In [3]:
# for preprocessing html data  
!pip3 install beautifulsoup4

# For RAG
!pip3 install langchain
!pip3 install langchainhub
!pip3 install chromadb
!pip3 install gpt4all

!pip3 install tqdm

# Needed if using LlamaCpp from LangChain
!pip3 install llama-cpp-python
!pip3 install pandas fuzzywuzzy sentence-transformers


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip3 install --upgrade pip
  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/e1/0c/ad295fd74bfac85358fd579e271cded3ac969de81f62dd0142c426b9da91/pandas-2.2.3-cp312-cp312-macosx_11_0_arm64.whl.metadata
  

In [6]:
import pandas as pd
from gpt4all import GPT4All
from fuzzywuzzy import process

In [51]:

def load_customer_support_data(csv_file):
    # Load the CSV data
    data = pd.read_csv(csv_file)
    return data

def preprocess_data(data):
    processed_data = data[['Ticket Subject', 'Ticket Description', 'Resolution']].dropna()
    
    processed_data['Ticket Subject'] = processed_data['Ticket Subject'].apply(lambda x: x.replace('"', '').replace("\n", "").replace("\t", "").strip())
    processed_data['Ticket Description'] = processed_data['Ticket Description'].apply(lambda x: x.replace('"', '').replace("\n", "").replace("\t", "").strip())
    processed_data['Resolution'] = processed_data['Resolution'].apply(lambda x: x.replace('"', '').replace("\n", "").replace("\t", "").strip())
    
    # Convert DataFrame to string format for saving to text file
    processed_output = processed_data.to_string(index=False)
    
    # Save to a text file
    with open('support_data.txt', 'w', encoding="utf-8") as f:
        f.write(processed_output)
    
    return processed_data


In [52]:
csv_path = 'customer_support_tickets.csv'
    
    # Load and preprocess the data
support_data = load_customer_support_data(csv_path)
processed_data = preprocess_data(support_data)

processed_data

,Ticket Subject,Ticket Description,Resolution
2,Network problem,I'm facing a problem with my {product_purchase...,Case maybe show recently my computer follow.
3,Account access,I'm having an issue with the {product_purchase...,Try capital clearly never color toward story.
4,Data loss,I'm having an issue with the {product_purchase...,West decision evidence bit.
10,Data loss,I'm having an issue with the {product_purchase...,Measure tonight surface feel forward.
11,Software bug,I'm having an issue with the {product_purchase...,Measure there house management pick knowledge ...
...,...,...,...
8452,Display issue,I'm having an issue with the {product_purchase...,Interesting show must successful.
8453,Peripheral compatibility,The {product_purchased} is unable to establish...,Skill expect admit and.
8455,Refund request,"I've recently set up my {product_purchased}, b...",Say position key appear behavior.
8466,Account access,I'm having an issue with the {product_purchase...,Eight account century nature kitchen.


In [53]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.llms import LlamaCpp
from langchain_core.output_parsers import StrOutputParser
from langchain.docstore.document import Document
from langchain import hub
from langchain_core.runnables import RunnablePassthrough

In [54]:
doc = Document(page_content=processed_data.to_string(index=False), 
                metadata={"source": "local"})

# split into different chunks
# chunk_size and chunk_overlap are a hyperparameters we choose 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=30)

# split the documents and convert to vector stores
all_splits = text_splitter.split_documents([doc])
vector_store = Chroma.from_documents(documents=all_splits, 
                                     embedding=GPT4AllEmbeddings())

In [55]:
from langchain_community.llms import GPT4All
#from gpt4all import GPT4All

llm = GPT4All(
    model="orca-mini-3b-gguf2-q4_0.gguf",
    max_tokens=2048, allow_download = True
)

In [56]:
llm.invoke("Cancellation request")

' form\nCancellation request form (PDF)'

In [57]:
# retrieve relevant docs
rag_prompt = hub.pull("rlm/rag-prompt")
retriever = vector_store.as_retriever()

# Create the langchain with retriever
qa_chain = (
    {"context": {}, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)
qa_chain.invoke("Cancellation request?")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langsmith/client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


" I'm sorry, but I need more information about your cancellation request to provide a specific answer. Can you please provide me with more details or context so that I can assist you better?"

In [58]:
# retrieve relevant docs
rag_prompt = hub.pull("rlm/rag-prompt")
rag_prompt.messages

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langsmith/client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]

In [59]:
def format_documents(documents):
    return "\n\n".join(doc.page_content for doc in documents)

In [60]:
retriever = vector_store.as_retriever()

# Create the langchain with retriever,
# prompt template and LLM
qa_chain = (
    {"context": retriever | format_documents, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)
qa_chain.invoke("Cancellation request?")

' \nWe are currently experiencing an issue with the product you have requested. Please try again later or contact our customer service for further assistance.'

In [61]:
retriever = vector_store.as_retriever()

# Create the langchain with retriever,
# prompt template and LLM
qa_chain = (
    {"context": retriever | format_documents, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)


In [62]:
qa_chain.invoke("there is an issue in product i recieved. can i get a full and proper refund?")

Exception ignored on calling ctypes callback function: <function LLModel._prompt_callback at 0x11e63c4a0>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gpt4all/_pyllmodel.py", line 614, in _prompt_callback
    @staticmethod

KeyboardInterrupt: 


''

In [64]:
qa_chain.invoke("How was password reset issue resolved?")

' It seems like you are experiencing an issue with the password reset option for your {product_purchased} account. We recommend contacting our customer support team at [email protected] and providing them with your account details so they can assist you further.'